In [6]:
from rdflib import Graph, ConjunctiveGraph, RDF, RDFS, OWL
from SPARQLWrapper import SPARQLWrapper2
import pandas as pds
import seaborn as sns

## define funciton for converting sparql results to a dataframe

In [ ]:
def sparql_to_df(results):
    data = {}
    for variable in results.variables:
        data[variable] = [x.value for x in results.getValues(variable)]
    return pds.DataFrame(data)

## set connection to local triple store

In [44]:
sparql = SPARQLWrapper2("http://192.168.1.243:7200/repositories/EDR_NO_ES")

## test getting some results

In [39]:
q = "select ?s ?p ?o where { ?s ?p ?o } limit 5"
sparql.setQuery(q)
res = sparql.query()
sparql_to_df(res)

,s,p,o
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#Thing
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#ObjectProperty
3,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2000/01/rdf-schema#Class
4,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#Class


## define some standard prefixes to use in queries

In [48]:
prefixes = """
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX part_of: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX has_part: <http://purl.obolibrary.org/obo/BFO_0000051>
PREFIX material: <http://purl.obolibrary.org/obo/OHD_0000000>
PREFIX tooth: <http://purl.obolibrary.org/obo/FMA_12516>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ada_num: <http://purl.obolibrary.org/obo/OHD_0000065>
PREFIX patient: <http://purl.obolibrary.org/obo/OHD_0000012>
PREFIX male_patient: <http://purl.obolibrary.org/obo/OHD_0000054>
PREFIX female_patient: <http://purl.obolibrary.org/obo/OHD_0000049>
PREFIX provider: <http://purl.obolibrary.org/obo/OHD_0000051>
PREFIX procedure: <http://purl.obolibrary.org/obo/OHD_0000002>
PREFIX restoration_procedure: <http://purl.obolibrary.org/obo/OHD_0000004>
PREFIX participates_in: <http://purl.obolibrary.org/obo/BFO_0000056>
PREFIX has_participant: <http://purl.obolibrary.org/obo/BFO_0000057>
PREFIX has_specified_input: <http://purl.obolibrary.org/obo/OBI_0000293>
PREFIX has_specified_output: <http://purl.obolibrary.org/obo/OBI_0000299>

"""

## find total number of patients that had a procedure

In [50]:
q = f"""
{prefixes}
select 
    (count(distinct ?patient) as ?patients)
    (count(distinct ?female) as ?females) 
    (count(distinct ?male) as ?males)  
where {{
    {{
      ?patient a patient: .
      ?tooth a tooth:; part_of: ?patient .
      ?proc a procedure:; has_specified_input: ?tooth . 
    }} union {{
      ?female a female_patient: .
      ?tooth a tooth:; part_of: ?female .
      ?proc a procedure:; has_specified_input: ?tooth  
    }} union {{
      ?male a male_patient: .
      ?tooth a tooth:; part_of: ?male .
      ?proc a procedure:; has_specified_input: ?tooth  
    }}
}}
"""

In [ ]:
sparql.setQuery(q)
res = sparql.query()
sparql_to_df(res)